# df_final 컬럼 설명
- channel_owner : 채널의 호스트
- channel_name : 채널명
- interval : 셀럽이 보낸 메시지 간격 (timestamp)
- interval_sec : 셀럽이 보낸 메시지 간격 (초단위)
- interval_mean : 셀럽이 보낸 메시지 간격 평균 (초단위)
- usage_time : 셀럽의 어플 사용시간 (초단위) (30분 이상 미사용시 사용 종료)
- period : 사용기간 (timestamp) (마지막으로 보낸 메시지 날짜 - 처음으로 보낸 메시지 날짜)
- period_sec : 사용기간 (초단위)
- first_send_date : 처음으로 보낸 메시지 날짜 (timestamp)
- last_send_date : 마지막으로 보낸 메시지 날짜 (timestamp)
- total_members : 채널에 들어와 있는 총 인원
- community_yn : 외부 커뮤니티 소유 여부 (1 / 0)
- personal_community_yn : 외부 커뮤니티 소유 여부 (1 / 0) (트게더, 아프리카 블로그 등 제외)
- message_count : 셀럽이 보낸 총 메시지 수
- message_count_per_period_sec : 사용기간 대비 메시지 수
- image_count : 셀럽이 보낸 이미지 수
- video_count : 셀럽이 보낸 영상 수
- sound_count : 셀럽이 보낸 음성 수
- total_sticker_price : 셀럽이 받는 샷(스티커) 총 합
- total_sticker_count : 셀럽이 받는 샷(스티커) 총 개수
- total_positive_sum : 전체 메시지 긍정지수(확률값) 총 합
- total_positive_count : 전체 메시지 긍정지수 총 개수
- celeb_positive_sum : 셀럽이 보낸 메시지 긍정지수(확률값) 총 합
- celeb_positive_count : 셀럽이 보낸 메시지 긍정지수 총 개수
- fan_positive_sum : 팬이 보낸 메시지 긍정지수(확률값) 총 합
- fan_positive_count : 팬이 보낸 메시지 긍정지수 총 개수
- twitch : 트위치 팔로워 수
- afreeca : 아프리카 애청자/팬클럽 수
- youtube : 유튜브 구독자 수
- recency : 2022-2-26과 최신 차이값 (일day단위)
- frequency : 셀럽이 보낸 총 메시지 수(message_count와 동일값이네요..?)

## 목표  
- 셀럽이 채팅방에 머문 시간
- 셀럽이 보낸 메세지 시간 간격
- 셀럽이 보낸 총 메시지
- 채팅방 총 인원
- 타 커뮤 사용 여부
- 첫 메시지 전송 일자 / 마지막 메시지 전송 일자
- 첫 메시지 전송 일자 대비 메세지 수
- 스티커 받은 금액/개수
- 메시지 감정(긍정) 수치(전체 / 셀럽이 보낸/ 팬이 보낸)
- 셉럽의 메시지 최근 정도 R  
- 셀럽의 메시지 빈도 F

In [ ]:
! pip install pickle-mixin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6006 sha256=fd0edaf025da6f8d40360c8a0038f4e55eb284ba7a9a71d1b79c25630ccd8801
  Stored in directory: /root/.cache/pip/wheels/2a/a4/6c/83bfbc3b94f1bb43d634b07a6a893fd437a45c58b29aea5142
Successfully built pickle-mixin


In [ ]:
# 워닝 은닉
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# 라이브러리 import
import pickle
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# columns 생략 없이 출력
pd.set_option('display.max_columns', None)
# 문자열 출력 길이
pd.options.display.max_colwidth = 100

In [ ]:
# 구글 드라이브 연걸
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# row 데이터 불러오기
with open('/content/drive/MyDrive/CP2/df_clean3.pickle', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data.columns

Index(['channel_url_x', 'type', 'message', 'created_at', 'id type',
       'channel_member_count_x', 'message_count', 'channel_created_at',
       'registration date', 'total_shot', '트위치', '아프리카(애청자/팬클럽)', '유튜브 수',
       '메인 플랫폼 링크', '콘텐츠 카테고리', '메인 평균 방송시간(9개)/업로드주기', '평균 시청 횟수/최근 조회수',
       '커뮤니티 링크', '성별', '얼굴 공개 여부', '채널주인', 'channel_name_secured',
       'sender_nickname_secured', 'sender_id_secured', 'sticker_name',
       'sticker_price', 'private_community', 'message_len'],
      dtype='object')

In [ ]:
# 원본 카피
df = data.copy()

In [ ]:
owner = df[df["채널주인"].notnull()]["채널주인"].unique()
nick = df[df["id type"] == "celeb"]["sender_nickname_secured"].unique()
channel_owner = df[(df["id type"] == "celeb") & (df["채널주인"].notnull())]["채널주인"].unique()
channel_owner2 = df[(df["id type"] == "fan") & (df["채널주인"].notnull())]["채널주인"].unique()
intersection = list(set(channel_owner) & set(channel_owner2))
print(len(owner)) # 전체 채널url 수
print(len(nick)) # 셀럽 닉네임 수
print(len(channel_owner)) # 셀럽 채널url 수
print(len(channel_owner2)) # 팬 채널url 수
print(len(intersection)) # 셀럽 & 팬 교집합 채널url 수

301
303
300
20
19


In [ ]:
# 닉네임을 기준으로 채널주인(url) 매칭 (셀럽 3명 url 미존재)
channel_host = df[(df["id type"] == "celeb") & (df["sender_nickname_secured"].notnull())]["sender_nickname_secured"].unique()
df_nick = pd.DataFrame()
nick_list = []
url_list = []
owner_url_list = []
for nick in channel_host:
    owner_url = df[(df["id type"] == "celeb") & (df["sender_nickname_secured"] == nick)]["채널주인"].unique()
    nick_list.append(nick)
    owner_url_list.append(owner_url[0])
df_nick["channel_owner"] = nick_list
df_nick["owner_url"] = owner_url_list

# 결측치 채우기
df_nick.fillna("None", inplace = True)

# 결측치 제거
# df_nick.dropna(inplace = True)

# 인덱스 초기화
df_nick.sort_values("channel_owner", inplace = True)
df_nick.reset_index(drop=True, inplace=True)

# channel_name_secured 매칭
channel_name_list = []
for i in range(len(df_nick)):
    check = df_nick.loc[i]
    owner = check[0]
    url = check[1]
    channel_name = df[(df["채널주인"] == df["channel_url_x"]) & (df["채널주인"] == url) & (df["sender_nickname_secured"] == owner)]["channel_name_secured"].unique()
    if len(channel_name) == 1:
        channel_name_list.append(channel_name[0])    
    elif len(channel_name) == 0:
        channel_name_list.append("None")
df_nick["channel_name"] = channel_name_list

df_nick.loc[df_nick['channel_owner'] == "하나*", 'channel_name'] = "하나*"
df_nick = df_nick[["channel_owner", "channel_name", "owner_url"]]

In [ ]:
# 메세지 시간 간격 테이블
df2 = pd.DataFrame(columns=["channel_owner", "interval", "interval_sec", "interval_mean"])
for n, owner in enumerate(df_nick["channel_owner"]):
    interval= []
    interval_sec = []
    # num = len(df[df["sender_nickname_secured"] == owner])
    num = len(df[(df["id type"] == "celeb") & (df["sender_nickname_secured"] == owner)])
    date_list = list(df[(df["id type"] == "celeb") & (df["sender_nickname_secured"] == owner)]["created_at"].sort_values())
    # date_list = list(df[df["sender_nickname_secured"] == owner]["created_at"].sort_values())
    for i in range(num):
        j = i+1
        if j < num:
            subtraction = date_list[j] - date_list[i]
            sec = subtraction.days*86400 + subtraction.seconds
            interval.append(subtraction)
            interval_sec.append(sec)
        if len(interval_sec) == 0:
            interval_mean = 0
        else:
            interval_mean = round(sum(interval_sec) / len(interval_sec))
    df2.loc[n] = [owner, interval, interval_sec, interval_mean]

# 인덱스 초기화
df2.sort_values("channel_owner", inplace = True)
df2.reset_index(drop=True, inplace=True)

# 채널 이름 추가
df2["channel_name"] = df_nick["channel_name"]

In [ ]:
# 셀럽이 보낸 총 메시지 수 : 보낸 사람 닉네임 기준
celeb_message_count = df[df["id type"] == "celeb"].groupby("sender_nickname_secured").count()
celeb_message_count.reset_index(drop=False, inplace=True)
cnt = celeb_message_count.message
df2["celeb_message_count"] = cnt

In [ ]:
# 팬이 보낸 총 메시지 수 : 채널 이름 기준
fan_message_count = df[df["id type"] == "fan"].groupby("channel_name_secured").agg({"message_count":"count"})
fan_message_count.reset_index(drop=False, inplace=True)

df2 = pd.merge(df2, fan_message_count, how="left", left_on='channel_name', right_on='channel_name_secured')
df2.rename(columns={"message_count":"fan_message_count"}, inplace=True)

In [ ]:
# 채팅방 총 인원
total_members = df[df["id type"] == "celeb"].groupby("sender_nickname_secured").mean()
total_members.reset_index(drop=False, inplace=True)
mean = total_members["channel_member_count_x"]
df2["total_members"] = mean

In [ ]:
# 셀럽의 앱 사용 시간
df2["usage_time"] = 0
for i in range(len(df2)):
    usage_time = []
    usage_time2 = []
    time = 0
    sec_list = df2["interval_sec"][i]
    for n, sec in enumerate(sec_list):
        if len(sec_list) == 1:
            if sec <= 1800:
                time = sec
                usage_time.append(time+1800)
            else:
                usage_time.append(3600)
            break

        if sec <= 1800:
            time = time + sec
            if n+1 == len(sec_list):
                usage_time.append(time+1800) 
        else:
            usage_time.append(time+1800)
            time = 0
            if n+1 == len(sec_list):
                usage_time.append(1800) 
    df2["usage_time"][i] = sum(usage_time)

In [ ]:
# 셀럽이 보낸 처음 & 마지막 날짜
df2["first_send_date"] = 0
df2["last_send_date"] = 0
for i, nick in enumerate(df_nick["channel_owner"]):
    min_date = df[df["sender_nickname_secured"] == nick]["created_at"].min()
    max_date = df[df["sender_nickname_secured"] == nick]["created_at"].max()
    df2["first_send_date"][i] = min_date
    df2["last_send_date"][i] = max_date

In [ ]:
# 타 커뮤니티 사용여부
community = df[df["id type"] == "celeb"].groupby("sender_nickname_secured").mean()
community.reset_index(drop=False, inplace=True)
community_yn = community["private_community"]
df2["private_community"] = community_yn

In [ ]:
# 기준날로부터 첫 메시지 전송일자 차이
print("첫 메시지 전송 날짜 : ", df["created_at"].min())
print("마지막 메시지 전송 날짜 : ", df["created_at"].max())
standard = datetime.strptime("2022-09-18 00:00:00", '%Y-%m-%d %H:%M:%S')

period = standard - df2["first_send_date"]
second = []
for i in range(len(period)):
    s = period[i].days*86400 + period[i].seconds
    second.append(s)
# df2["period"] = period
df2["period_sec"] = second

첫 메시지 전송 날짜 :  2022-02-27 01:03:05
마지막 메시지 전송 날짜 :  2022-09-17 23:59:49


In [ ]:
# 1days 기준 가입일자(?) or 전송기간 대비 메세지 수
df2["celeb_message_count_per_period_sec"] = df2["celeb_message_count"] / (df2["period_sec"] / 86400)

In [ ]:
# 받은 스티커 총합 / 총개수
df_merge = pd.DataFrame()
sticker_sum = df.groupby("channel_name_secured").sum()
sticker_count = df.groupby("channel_name_secured").count()
sticker_sum.reset_index(drop=False, inplace=True)
sticker_count.reset_index(drop=False, inplace=True)

s_sum = sticker_sum[["channel_name_secured", "sticker_price"]]
s_count = sticker_count[["channel_name_secured", "sticker_name"]]

df_merge = pd.merge(df2, s_sum, how='left', left_on='channel_name', right_on='channel_name_secured')
df_merge = pd.merge(df_merge, s_count, how='left', left_on='channel_name', right_on='channel_name_secured')

In [ ]:
# 감정분석 데이터 불러오기
with open('/content/drive/MyDrive/CP2/df_sentiment.pickle', 'rb') as f:
    data_sentiment = pickle.load(f)

In [ ]:
# 원본 카피
df_sentiment = data_sentiment.copy()

# 긍정 확률값 컬럼 생성
def positive_score(dic):
    return dic['positive']
df_sentiment['positive_score'] = df_sentiment['labels_sh'].apply(positive_score)
def negative_score(dic):
    return dic['negative']
df_sentiment['negative_score'] = df_sentiment['labels_sh'].apply(negative_score)

columns = ['type', 'message', 'created_at', 'id type', 'channel_name_secured', 'sender_nick_id', 'LEN', 'labels_sh', 'positive_score', 'negative_score']
df_sentiment = df_sentiment[columns]

# 파일 타입 제거
File = df_sentiment[df_sentiment["type"] == "FILE"].index
df_sentiment.drop(File, inplace=True)

In [ ]:
# 전체/셀럽/팬의 감정분석 긍정 총합/건수
positive_sum = df_sentiment.groupby(["channel_name_secured"]).agg({"positive_score":"sum"})
positive_sum.reset_index(drop=False, inplace=True)
positive_count = df_sentiment.groupby(["channel_name_secured"]).agg({"positive_score":"count"})
positive_count.reset_index(drop=False, inplace=True)

celeb_positive_sum = df_sentiment[df_sentiment["id type"] == "celeb"].groupby(["channel_name_secured"]).agg({"positive_score":"sum"})
celeb_positive_sum.reset_index(drop=False, inplace=True)
celeb_positive_count = df_sentiment[df_sentiment["id type"] == "celeb"].groupby(["channel_name_secured"]).agg({"positive_score":"count"})
celeb_positive_count.reset_index(drop=False, inplace=True)

fan_positive_sum = df_sentiment[df_sentiment["id type"] == "fan"].groupby(["channel_name_secured"]).agg({"positive_score":"sum"})
fan_positive_sum.reset_index(drop=False, inplace=True)
fan_positive_count = df_sentiment[df_sentiment["id type"] == "fan"].groupby(["channel_name_secured"]).agg({"positive_score":"count"})
fan_positive_count.reset_index(drop=False, inplace=True)

positive_merge = pd.merge(positive_sum, positive_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
positive_merge.rename(columns={"channel_name_secured" : "channel_name", "positive_score_x" : "total_positive_sum", "positive_score_y" : "total_positive_count"}, inplace=True)

celeb_positive_merge = pd.merge(celeb_positive_sum, celeb_positive_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
celeb_positive_merge.rename(columns={"channel_name_secured" : "channel_name", "positive_score_x" : "celeb_positive_sum", "positive_score_y" : "celeb_positive_count"}, inplace=True)

fan_positive_merge = pd.merge(fan_positive_sum, fan_positive_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
fan_positive_merge.rename(columns={"channel_name_secured" : "channel_name", "positive_score_x" : "fan_positive_sum", "positive_score_y" : "fan_positive_count"}, inplace=True)

df_merge = pd.merge(df_merge, positive_merge, how='left', left_on='channel_name', right_on='channel_name')
df_merge = pd.merge(df_merge, celeb_positive_merge, how='left', left_on='channel_name', right_on='channel_name')
df_merge = pd.merge(df_merge, fan_positive_merge, how='left', left_on='channel_name', right_on='channel_name')

In [ ]:
# 전체/셀럽/팬의 감정분석 부정 총합/건수
negative_sum = df_sentiment.groupby(["channel_name_secured"]).agg({"negative_score":"sum"})
negative_sum.reset_index(drop=False, inplace=True)
negative_count = df_sentiment.groupby(["channel_name_secured"]).agg({"negative_score":"count"})
negative_count.reset_index(drop=False, inplace=True)

celeb_negative_sum = df_sentiment[df_sentiment["id type"] == "celeb"].groupby(["channel_name_secured"]).agg({"negative_score":"sum"})
celeb_negative_sum.reset_index(drop=False, inplace=True)
celeb_negative_count = df_sentiment[df_sentiment["id type"] == "celeb"].groupby(["channel_name_secured"]).agg({"negative_score":"count"})
celeb_negative_count.reset_index(drop=False, inplace=True)

fan_negative_sum = df_sentiment[df_sentiment["id type"] == "fan"].groupby(["channel_name_secured"]).agg({"negative_score":"sum"})
fan_negative_sum.reset_index(drop=False, inplace=True)
fan_negative_count = df_sentiment[df_sentiment["id type"] == "fan"].groupby(["channel_name_secured"]).agg({"negative_score":"count"})
fan_negative_count.reset_index(drop=False, inplace=True)

negative_merge = pd.merge(negative_sum, negative_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
negative_merge.rename(columns={"channel_name_secured" : "channel_name", "negative_score_x" : "total_negative_sum", "negative_score_y" : "total_negative_count"}, inplace=True)

celeb_negative_merge = pd.merge(celeb_negative_sum, celeb_negative_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
celeb_negative_merge.rename(columns={"channel_name_secured" : "channel_name", "negative_score_x" : "celeb_negative_sum", "negative_score_y" : "celeb_negative_count"}, inplace=True)

fan_negative_merge = pd.merge(fan_negative_sum, fan_negative_count, how='left', left_on='channel_name_secured', right_on='channel_name_secured')
fan_negative_merge.rename(columns={"channel_name_secured" : "channel_name", "negative_score_x" : "fan_negative_sum", "negative_score_y" : "fan_negative_count"}, inplace=True)

df_merge = pd.merge(df_merge, negative_merge, how='left', left_on='channel_name', right_on='channel_name')
df_merge = pd.merge(df_merge, celeb_negative_merge, how='left', left_on='channel_name', right_on='channel_name')
df_merge = pd.merge(df_merge, fan_negative_merge, how='left', left_on='channel_name', right_on='channel_name')

df_merge["sentiment"] = df_merge["total_positive_sum"] - df_merge["total_negative_sum"]
df_merge["celeb_sentiment"] = df_merge["celeb_positive_sum"] - df_merge["celeb_negative_sum"]
df_merge["fan_sentiment"] = df_merge["fan_positive_sum"] - df_merge["fan_negative_sum"]

In [ ]:
# 트위치, 아프리카, 유튜브 팔로워 수
follow = df[df["id type"] == "celeb"].groupby(["sender_nickname_secured"]).agg({"트위치" : "max", "아프리카(애청자/팬클럽)" : "max", "유튜브 수" : "max"})
follow.reset_index(drop=False, inplace=True)
follow.columns = ['sender_nickname', 'twitch','afreeca','youtube']
# follow.fillna(0, inplace=True)
df_merge = pd.merge(df_merge, follow, how='left', left_on='channel_owner', right_on='sender_nickname')

In [ ]:
# 셀럽이 보낸 이미지 수
image = df[(df["id type"] == "celeb") & (df["message"].str.contains(".jpe|.jpg|.png"))].groupby("sender_nickname_secured").agg({"message" : "count"})
image.reset_index(drop=False, inplace=True)
image.rename(columns={"message" : "image_count"}, inplace=True)

# 셀럽이 보낸 영상 수
video = df[(df["id type"] == "celeb") & (df["message"].str.contains(".mp4|.mov"))].groupby("sender_nickname_secured").agg({"message" : "count"})
video.reset_index(drop=False, inplace=True)
video.rename(columns={"message" : "video_count"}, inplace=True)

# 셀럽이 보낸 음성 수
sound = df[(df["id type"] == "celeb") & (df["message"].str.contains(".wav"))].groupby("sender_nickname_secured").agg({"message" : "count"})
sound.reset_index(drop=False, inplace=True)
sound.rename(columns={"message" : "sound_count"}, inplace=True)

# merge
file_merge = pd.merge(image, video, how='left', on='sender_nickname_secured')
file_merge = pd.merge(file_merge, sound, how='left', on='sender_nickname_secured')

df_merge = pd.merge(df_merge, file_merge, how='left', left_on='channel_owner', right_on='sender_nickname_secured')

In [ ]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 302
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   channel_owner                       303 non-null    object 
 1   interval                            303 non-null    object 
 2   interval_sec                        303 non-null    object 
 3   interval_mean                       303 non-null    object 
 4   channel_name                        303 non-null    object 
 5   celeb_message_count                 303 non-null    int64  
 6   channel_name_secured_x              224 non-null    object 
 7   fan_message_count                   224 non-null    float64
 8   total_members                       303 non-null    float64
 9   usage_time                          303 non-null    int64  
 10  first_send_date                     303 non-null    object 
 11  last_send_date                      303 non-n

In [ ]:
df_merge.fillna(0, inplace = True)

type_change = {'interval_mean':'int', 'total_members':'int', 'image_count':'int', 'video_count':'int', 'sound_count':'int', 'sticker_price':'int', 'twitch':'int', 'youtube':'int', 'private_community' : 'str'}
df_merge = df_merge.astype(type_change)

# 메세지:파일 타입 총 합
df_merge["file_count"] = df_merge["image_count"] + df_merge["video_count"] + df_merge["sound_count"]

# 팔로워 총 합
follow_count = []
for i in range(len(df_merge)):
    twitch = df_merge["twitch"][i]
    afreeca = df_merge["afreeca"][i]
    youtube = df_merge["youtube"][i]
    follow = max(twitch, afreeca, youtube)
    follow_count.append(follow)
df_merge["follow_count"] = follow_count

In [ ]:
# 컬럼명 / 컬럼 순서 정리 (최종)
columns = ['channel_owner', 'channel_name', 
           'interval', 'interval_sec', 'interval_mean', 
           'usage_time', 'period_sec', 'first_send_date', 'last_send_date', 'private_community',  
           'total_members', 'celeb_message_count', 'celeb_message_count_per_period_sec', 'fan_message_count',
           'file_count', 'image_count', 'video_count', 'sound_count', 'sticker_price', 'sticker_name', 
           'total_positive_sum', 'total_positive_count', 'celeb_positive_sum', 'celeb_positive_count', 'fan_positive_sum', 'fan_positive_count', 
           'total_negative_sum', 'total_negative_count', 'celeb_negative_sum', 'celeb_negative_count', 'fan_negative_sum', 'fan_negative_count',
           'sentiment', 'celeb_sentiment', 'fan_sentiment', 'follow_count', 'twitch', 'afreeca', 'youtube']
df_final = df_merge[columns]
df_final.rename(columns={"sticker_price" : "total_sticker_price", "sticker_name" : "total_sticker_count"}, inplace=True)

In [ ]:
# 시각화 용 csv 저장
df_final.to_csv('/content/drive/MyDrive/CP2/df_final2.csv', index=False, encoding="utf-8-sig")

# df_final 컬럼 설명
- channel_owner : 채널의 호스트
- channel_name : 채널명
- interval : 셀럽이 보낸 메시지 간격 (timestamp)
- interval_sec : 셀럽이 보낸 메시지 간격 (초단위)
- interval_mean : 셀럽이 보낸 메시지 간격 평균 (초단위)
- usage_time : 셀럽의 어플 사용시간 (초단위) (30분 이상 미사용시 사용 종료)
- period : 사용기간 (timestamp) (마지막으로 보낸 메시지 날짜 - 처음으로 보낸 메시지 날짜)
- period_sec : 사용기간 (초단위)
- first_send_date : 처음으로 보낸 메시지 날짜 (timestamp)
- last_send_date : 마지막으로 보낸 메시지 날짜 (timestamp)
- total_members : 채널에 들어와 있는 총 인원
- community_yn : 외부 커뮤니티 소유 여부 (1 / 0)
- personal_community_yn : 외부 커뮤니티 소유 여부 (1 / 0) (트게더, 아프리카 블로그 등 제외)
- message_count : 셀럽이 보낸 총 메시지 수
- message_count_per_period_sec : 사용기간 대비 메시지 수
- image_count : 셀럽이 보낸 이미지 수
- video_count : 셀럽이 보낸 영상 수
- sound_count : 셀럽이 보낸 음성 수
- total_sticker_price : 셀럽이 받는 샷(스티커) 총 합
- total_sticker_count : 셀럽이 받는 샷(스티커) 총 개수
- total_positive_sum : 전체 메시지 긍정지수(확률값) 총 합
- total_positive_count : 전체 메시지 긍정지수 총 개수
- celeb_positive_sum : 셀럽이 보낸 메시지 긍정지수(확률값) 총 합
- celeb_positive_count : 셀럽이 보낸 메시지 긍정지수 총 개수
- fan_positive_sum : 팬이 보낸 메시지 긍정지수(확률값) 총 합
- fan_positive_count : 팬이 보낸 메시지 긍정지수 총 개수
- twitch : 트위치 팔로워 수
- afreeca : 아프리카 애청자/팬클럽 수
- youtube : 유튜브 구독자 수
- recency : 2022-2-26과 최신 차이값 (일day단위)
- frequency : 셀럽이 보낸 총 메시지 수(message_count와 동일값이네요..?)